# Annotation Files for Manual Linking
>This notebook facilitates writing out non-linked players to an annotation file - then handling the write out to annotation files.  The files can then be loaded here to process: https://j535d165.github.io/recordlinkage-annotator/  Finally the result file can be loaded back into the j_notebook directory where it is picked up here and added to the Record Links database table.

In [11]:
import pandas as pd
import recordlinkage as rl
from recordlinkage.index import Block
import sqlite3 as sql
import functions as fx
import core_constants as cc
from recordlinkage.base import BaseCompareFeature

class CompareNonExactYears(BaseCompareFeature):

    def _compute_vectorized(self, s1, s2):
        """Compare zipcodes.

        If the zipcodes in both records are identical, the similarity
        is 0. If the first two values agree and the last two don't, then
        the similarity is 0.5. Otherwise, the similarity is 0.
        """

        # check if the zipcode are identical (return 1 or 0)
        sim = (((s2 + 3) == s1) | ((s2 + 4) == s1) | ((s2 + 5) == s1)).astype(float)

        # check the first 2 numbers of the distinct comparisons
        sim[(sim == 0) & (((s1 == (s2 + 2))) | (s1 == (s2 + 6)))] = 0.25

        return (sim)

## Load Datasets
> Load the source and target datasets.  Currently it is only written for 247 and Rivals but needs to be extended

In [12]:
keydatasets = [1,3]
dataset_names = []
for ds in keydatasets:
    SQL = ''' SELECT * FROM DataSet'''
    datasets = (fx.connDBAndReturnDF(SQL)).to_dict('records')
    dataset = next(item for item in datasets if item["KeyDataSet"] == ds)
    SQL = """SELECT * FROM {}""".format(dataset['UnlinkedView'])
    
    if (ds == 1 or ds == 2):
        indexDs = 'IDYR'
    else:
        indexDs = 'ID'
    vars()[dataset['DataSet']] = (fx.connDBAndReturnDF(SQL)).set_index(indexDs)
    (vars()[dataset['DataSet']]).index.name = dataset['DataSet'] + indexDs
    (vars()[dataset['DataSet']])[indexDs] = (vars()[dataset['DataSet']]).index.get_level_values(0)
    dataset_names.append(vars() [dataset['DataSet']])

In [13]:
SQL = ''' SELECT TargetFieldName FROM Features where KeyMasterDataSet = {} and KeyTargetDataSet = {} and MatchType = "block"'''.format(keydatasets[0], keydatasets[1])
blockersRaw = (fx.connDBAndReturnDF(SQL)).to_dict('records')
blockers = []
for i in blockersRaw:
    for key,value in i.items():
        blockers.append(value)

indexer = rl.BlockIndex(on = blockers)
candidate_links = indexer.index(dataset_names[1], dataset_names[0])

In [14]:
c = rl.Compare()

SQL = '''SELECT TargetFieldName, MatchType, Method FROM Features where KeyMasterDataSet = {} and KeyTargetDataSet = {} and MatchType <> "block"'''.format(keydatasets[0], keydatasets[1])
featureFields = (fx.connDBAndReturnDF(SQL).to_dict('records'))

for feature in featureFields:
    
    if (feature['MatchType'] == 'exact'):
        c.exact(feature['TargetFieldName'], feature['TargetFieldName'], label = feature['TargetFieldName'])
    elif(feature['MatchType'] == 'string' and feature['Method'] is None):
        c.string(feature['TargetFieldName'], feature['TargetFieldName'], label = feature['TargetFieldName'])
    elif(feature['MatchType'] == 'string' and feature['Method'] is not None):
        c.string(feature['TargetFieldName'], feature['TargetFieldName'], method = feature['Method'], label = feature['TargetFieldName'])
    elif(feature['MatchType'] == 'numeric' and feature['Method'] is not None):
        c.numeric(feature['TargetFieldName'], feature['TargetFieldName'], method = feature['Method'], label = feature['TargetFieldName'])
    elif(feature['MatchType'] == 'custom_year' and feature['Method'] is None):
        c.add(CompareNonExactYears(feature['TargetFieldName'], feature['TargetFieldName'], label = feature['TargetFieldName']))
try:
    features = c.compute(candidate_links, dataset_names[1], dataset_names[0])
except KeyError as e:
    print(e)

In [15]:
sum = 0
for column in features:
    sum = sum + features[column]

print(len(featureFields))
features['Sum'] = sum/len(featureFields)

features.head()

features.to_csv('test_features.csv')

4


In [ ]:
count = 0
filteredList = []
noMatch = []

features['nfl_ID'] = features.index.get_level_values(0)
features['sports247_IDYR'] = features.index.get_level_values(1)


for idx, data in features.groupby(level=0):

    data = data.loc[data['Sum'].idxmax()]
    if (data['ID'] == 1):
        filteredList.append(data)
    elif (data['ID'] != 1 and data['Sum'] > .1):
        filteredList.append(data)
    else:
        noMatch.append(data)

In [ ]:
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.head()
dfFinal.sort_values(by='Sum', ascending=False)

dfFinal.to_csv('test_features.csv')

## HEY ANDREW

> Your results are weird because you can't block on a dynamic Year + 5 value before doing a max.  Or at least that's the though.  The number of NFL Draftees, All Americans, etc is much smaller than the overall universe though, so you don't need to be perfect.  Just find something generic that works and knock it out.

## Create Annotation file
> This is actually terrible code.  Currently I'm manually updating by length.  Need to loop through.  TODO

In [ ]:
dfFinal.shape[0]

In [ ]:
# Need to look at how this worked before.  You broke something you idiot.
length = dfFinal.shape[0]
count = 0
file_no = 1
while (count < length):
    rl.write_annotation_file(
        "..//Annotations//Annotations//annotate_NFL" + str(file_no) + ".json",
        dfFinal[count + 1:count + 500],
        dataset_names[1],
        dataset_names[0],
        dataset_a_name="247 Sports",
        dataset_b_name="NFLData")
    count = count + 500
    file_no = file_no + 1

## Read in Result
> Once you handled the above file in the annotator, the below code will verify that the result file is correctly located in the j_notebooks folder and will read it into a flat index - which makes it easier to insert into the db.

In [ ]:
annotation = rl.read_annotation_file("..//Annotations//Results//result_4.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

## Insert into the DB

In [ ]:
for record in annotation_dict:
    Values = [record[0], record[1], 2, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()